In [1]:
! pip install cvxopt
! pip install cvxpy
! pip install -U pymoo==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.0/299.0 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.4/706.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 4.2 MB/s eta 0:00:00
  Created wheel for pymoo: filename=pymoo-0.5.0-cp310-cp310-linux_x86_64.whl size=1189775 sha256=0f18a3a312ee2cc180d4c6ef01a229c5cdaeac0405db6b326fb288b4ab90cd4b
  Stored in directory: /root/.cache/pip/wheels/bd/81/54/0ea5fd29acdfbbe

In [2]:
import logging
import argparse
from collections import defaultdict
import time
import numpy as np
import torch
from torch import nn
from tqdm import trange
import os
import sys

sys.path.append("/kaggle/input/hyper-trans/Hyper_trans/MTL/experiments/Multi_task/MultiMNIST")
print(os.getcwd())
from metrics import hypervolumn
from data import Dataset
from models.hyper_mlp import (
    LeNetHyper,
    LeNetTarget,
)
from models.hyper_trans import (
    LeNetHyper_trans,
    LeNetTarget_trans,
)
from utils import (
    circle_points,
    count_parameters,
    set_logger,
    set_seed,
)
from solver import EPOSolver, LinearScalarizationSolver, ChebyshevBasedSolver, UtilityBasedSolver

/kaggle/working


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


(CVXPY) May 16 04:13:52 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Version of ortools (9.4.1874) is too old. Expected >= 9.5.0.')
(CVXPY) May 16 04:13:52 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Version of ortools (9.4.1874) is too old. Expected >= 9.7.0.')


In [3]:
from typing import List

import torch
import torch.nn.functional as F
from torch import nn
import torchvision.models as models
import math
class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, n_head):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.attention = ScaleDotProductAttention()
        self.w_q = nn.Linear(d_model, self.n_head)
        self.w_k = nn.Linear(d_model, self.n_head)
        self.w_v = nn.Linear(d_model, self.n_head)
        self.w_concat = nn.Linear(self.n_head, d_model)

    def forward(self, q, k, v, mask=None):
        # 1. dot product with weight matrices
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)

        # 2. split tensor by number of heads
        q, k, v = self.split(q), self.split(k), self.split(v)

        # 3. do scale dot product to compute similarity
        out, attention = self.attention(q, k, v, mask=mask)
        
        # 4. concat and pass to linear layer
        out = self.concat(out)
        out = self.w_concat(out)

        # 5. visualize attention map
        # TODO : we should implement visualization

        return out

    def split(self, tensor):
        """
        split tensor by number of head

        :param tensor: [batch_size, length, d_model]
        :return: [batch_size, head, length, d_tensor]
        """
        batch_size, length, d_model = tensor.size()

        d_tensor = d_model // self.n_head
        tensor = tensor.view(batch_size, length, self.n_head, d_tensor).transpose(1, 2)
        # it is similar with group convolution (split by number of heads)

        return tensor

    def concat(self, tensor):
        """
        inverse function of self.split(tensor : torch.Tensor)

        :param tensor: [batch_size, head, length, d_tensor]
        :return: [batch_size, length, d_model]
        """
        batch_size, head, length, d_tensor = tensor.size()
        d_model = head * d_tensor

        tensor = tensor.transpose(1, 2).contiguous().view(batch_size, length, d_model)
        return tensor
class ScaleDotProductAttention(nn.Module):
    """
    compute scale dot product attention

    Query : given sentence that we focused on (decoder)
    Key : every sentence to check relationship with Qeury(encoder)
    Value : every sentence same with Key (encoder)
    """

    def __init__(self):
        super(ScaleDotProductAttention, self).__init__()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, mask=None, e=1e-12):
        # input is 4 dimension tensor
        # [batch_size, head, length, d_tensor]
        batch_size, head, length, d_tensor = k.size()

        # 1. dot product Query with Key^T to compute similarity
        k_t = k.transpose(2, 3)  # transpose
        score = (q @ k_t) / math.sqrt(d_tensor)  # scaled dot product

        # 2. apply masking (opt)
        if mask is not None:
            score = score.masked_fill(mask == 0, -10000)

        # 3. pass them softmax to make [0, 1] range
        score = self.softmax(score)

        # 4. multiply with Value
        v = score @ v

        return v, score
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-12):
        super(LayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(d_model))
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, unbiased=False, keepdim=True)
        # '-1' means last dimension. 

        out = (x - mean) / torch.sqrt(var + self.eps)
        out = self.gamma * out + self.beta
        return out
class Attention(nn.Module):
    def __init__(self, num_heads, ray_hidden_dim):
        super(Attention, self).__init__()
        self.num_heads = num_heads
        self.ray_hidden_dim = ray_hidden_dim
        self.multi_head_attention = nn.MultiheadAttention(embed_dim=self.ray_hidden_dim, num_heads=self.num_heads)
        self.ffn1 = nn.Linear(self.ray_hidden_dim, self.ray_hidden_dim)
        self.ffn2 = nn.Linear(self.ray_hidden_dim, self.ray_hidden_dim)
    def forward(self, x):
        x_ = x
        x = x.unsqueeze(1)
        x,_ = self.multi_head_attention(x,x,x)
        x = x.squeeze(1)
        x = x + x_
        x_ = x
        x = self.ffn1(x)
        x = F.gelu(x)
        x = self.ffn2(x)
        x = x + x_
        return x
class Encode(nn.Module):
    def __init__(self, n_tasks, ray_hidden_dim):
        super(Encode, self).__init__()
        self.n_tasks = n_tasks
        self.ray_hidden_dim = ray_hidden_dim
        if self.n_tasks == 2:
            self.embedding_layer1 =  nn.Sequential(nn.Linear(1, self.ray_hidden_dim),nn.ReLU(inplace=True))
            self.embedding_layer2 =  nn.Sequential(nn.Linear(1, self.ray_hidden_dim),nn.ReLU(inplace=True))
        else:
            self.embedding_layer1 =  nn.Sequential(nn.Linear(1, self.ray_hidden_dim),nn.ReLU(inplace=True))
            self.embedding_layer2 =  nn.Sequential(nn.Linear(1, self.ray_hidden_dim),nn.ReLU(inplace=True))
            self.embedding_layer3 =  nn.Sequential(nn.Linear(1, self.ray_hidden_dim),nn.ReLU(inplace=True))
    def forward(self, ray):
        if ray.shape[0] == 2:
            x = torch.stack((self.embedding_layer1(ray[0].unsqueeze(0)),self.embedding_layer2(ray[1].unsqueeze(0))))
        else:
            x = torch.stack((self.embedding_layer1(ray[0].unsqueeze(0)),self.embedding_layer2(ray[1].unsqueeze(0)),self.embedding_layer3(ray[2].unsqueeze(0))))
        return x
class Decode(nn.Module):
    def __init__(self, out_dim,n_tasks):
        super(Decode, self).__init__()
        self.out_dim = out_dim
        self.out_task = nn.Linear(n_tasks, 1)
    def forward(self, x):
        x = self.out_task(x)
        return x
# class Hyper_trans3(nn.Module):
#       "Hypernetwork"

#       def __init__(self, ray_hidden_dim=100, out_dim = 2,n_tasks=2,num_hidden_layer=2,last_activation='relu'):
#             super().__init__()
#             self.out_dim = out_dim
#             self.n_tasks = n_tasks
#             self.ray_hidden_dim = ray_hidden_dim
#             self.num_hidden_layer = num_hidden_layer
#             self.last_activation = last_activation
#             self.encoder = Encode(n_tasks, ray_hidden_dim)
#             self.decoder = Decode(out_dim, n_tasks)
#             self.output_layer =  nn.Linear(self.ray_hidden_dim, self.out_dim)
#             self.attention = Attention(1,self.ray_hidden_dim)
#       def forward(self, ray):
#             x = self.encoder(ray)
#             x =  self.attention(x)
#             x = self.output_layer(x)
#             x = x.permute(1,2,0)
#             x = self.decoder(x)
#             x = x.squeeze(2)
#             if self.last_activation == 'relu':
#                 x = F.relu(x)
#             elif self.last_activation == 'sigmoid':
#                 x = F.sigmoid(x)
#             elif self.last_activation == 'softmax':
#                 x = F.softmax(x)    
#             else:
#                 x = x
#             return x
class LeNetHyper_trans(nn.Module):
    """LeNet Hypernetwork"""

    def __init__(
        self,
        kernel_size: List[int],
        ray_hidden_dim=100,
        out_dim=10,
        target_hidden_dim=50,
        n_kernels=10,
        n_conv_layers=2,
        n_hidden=1,
        n_tasks=2,
        act_type = 'relu'
    ):
        super().__init__()
        self.n_conv_layers = n_conv_layers
        self.n_hidden = n_hidden
        self.n_tasks = n_tasks
        self.act_type = act_type
        assert len(kernel_size) == n_conv_layers, (
            "kernel_size is list with same dim as number of "
            "conv layers holding kernel size for each conv layer"
        )

        # self.ray_mlp = nn.Sequential(
        #     nn.Linear(2, ray_hidden_dim),
        #     nn.ReLU(inplace=True),
        #     nn.Linear(ray_hidden_dim, ray_hidden_dim),
        #     nn.ReLU(inplace=True),
        #     nn.Linear(ray_hidden_dim, ray_hidden_dim),
        # )
        #self.output_layer =  nn.Linear(ray_hidden_dim, n_kernels * kernel_size[0] * kernel_size[0])

        if self.n_tasks == 2:
            self.embedding_layer1 =  nn.Sequential(nn.Linear(1, ray_hidden_dim),nn.ReLU(inplace=True))
            self.embedding_layer2 =  nn.Sequential(nn.Linear(1, ray_hidden_dim),nn.ReLU(inplace=True))
        else:
            self.embedding_layer1 =  nn.Sequential(nn.Linear(1, ray_hidden_dim),nn.ReLU(inplace=True))
            self.embedding_layer2 =  nn.Sequential(nn.Linear(1, ray_hidden_dim),nn.ReLU(inplace=True))
            self.embedding_layer3 =  nn.Sequential(nn.Linear(1, ray_hidden_dim),nn.ReLU(inplace=True))
        self.attention = nn.MultiheadAttention(embed_dim=ray_hidden_dim, num_heads=1) #MultiHeadAttention(d_model=ray_hidden_dim, n_head=2) #nn.MultiheadAttention(embed_dim=ray_hidden_dim, num_heads=2)
        #self.pos_embedding = nn.Parameter(torch.randn(self.n_tasks, ray_hidden_dim))
        self.ffn1 = nn.Linear(ray_hidden_dim,ray_hidden_dim)
        self.ffn2 = nn.Linear(ray_hidden_dim,ray_hidden_dim)
        
        #self.ffn3 = nn.Linear(self.n_tasks,1)
        self.output_layer = nn.Linear(ray_hidden_dim, ray_hidden_dim) #nn.Sequential(nn.Linear(ray_hidden_dim,ray_hidden_dim), nn.ReLU(inplace=True))  #nn.Linear(ray_hidden_dim, ray_hidden_dim)
        
        # self.encoder = Encode(n_tasks, ray_hidden_dim)
        # self.decoder = Decode(out_dim, n_tasks)
        # self.output_layer =  nn.Linear(ray_hidden_dim, ray_hidden_dim)
        # self.attention = Attention(1,ray_hidden_dim)

        self.conv_0_weights = nn.Linear(
            ray_hidden_dim, n_kernels * kernel_size[0] * kernel_size[0]
        )
        self.conv_0_bias = nn.Linear(ray_hidden_dim, n_kernels)

        for i in range(1, n_conv_layers):
            # previous number of kernels
            p = 2 ** (i - 1) * n_kernels
            # current number of kernels
            c = 2 ** i * n_kernels

            setattr(
                self,
                f"conv_{i}_weights",
                nn.Linear(ray_hidden_dim, c * p * kernel_size[i] * kernel_size[i]),
            )
            setattr(self, f"conv_{i}_bias", nn.Linear(ray_hidden_dim, c))

        latent = 25
        self.hidden_0_weights = nn.Linear(
            ray_hidden_dim, target_hidden_dim * 2 ** i * n_kernels * latent
        )
        self.hidden_0_bias = nn.Linear(ray_hidden_dim, target_hidden_dim)

        for j in range(n_tasks):
            setattr(
                self,
                f"task_{j}_weights",
                nn.Linear(ray_hidden_dim, target_hidden_dim * out_dim),
            )
            setattr(self, f"task_{j}_bias", nn.Linear(ray_hidden_dim, out_dim))

    def shared_parameters(self):
        return list([p for n, p in self.named_parameters() if "task" not in n])

    def forward(self, ray):

        if ray.shape[0] == 2:
            x = torch.stack((self.embedding_layer1(ray[0].unsqueeze(0)),self.embedding_layer2(ray[1].unsqueeze(0))))
        else:
            x = torch.stack((self.embedding_layer1(ray[0].unsqueeze(0)),self.embedding_layer2(ray[1].unsqueeze(0)),self.embedding_layer3(ray[2].unsqueeze(0))))
        #x += self.pos_embedding[:,:]
        x_ = x
        x = x.unsqueeze(1)
        x,_ = self.attention(x,x,x)
        x = x.squeeze(1)
        x = x + x_
        x_ = x
        x = self.ffn1(x)
        if self.act_type == 'relu':
            x = F.relu(x) 
        else:
            x = F.gelu(x)
        x = self.ffn2(x)
        x = x + x_
        x = self.output_layer(x)
        x = F.relu(x) 
        # print(x.shape)
        features = torch.mean(x,dim=0)
        # print(features.shape)
        # x = self.encoder(ray)
        # x =  self.attention(x)
        # x = self.output_layer(x)
        # x = F.relu(x) 
        # x = x.permute(1,0)
        # x = self.decoder(x)
        # #print(x.shape)
        # features = x.squeeze(1)
        
        

        out_dict = {}
        layer_types = ["conv", "hidden", "task"]

        for i in layer_types:
            if i == "conv":
                n_layers = self.n_conv_layers
            elif i == "hidden":
                n_layers = self.n_hidden
            elif i == "task":
                n_layers = self.n_tasks

            for j in range(n_layers):
                out_dict[f"{i}{j}.weights"] = getattr(self, f"{i}_{j}_weights")(
                    features
                )
                out_dict[f"{i}{j}.bias"] = getattr(self, f"{i}_{j}_bias")(
                    features
                ).flatten()

        return out_dict


class LeNetTarget_trans(nn.Module):
    """LeNet target network"""

    def __init__(
        self,
        kernel_size,
        n_kernels=10,
        out_dim=10,
        target_hidden_dim=50,
        n_conv_layers=2,
        n_tasks=2,
    ):
        super().__init__()
        assert len(kernel_size) == n_conv_layers, (
            "kernel_size is list with same dim as number of "
            "conv layers holding kernel size for each conv layer"
        )
        self.n_kernels = n_kernels
        self.kernel_size = kernel_size
        self.out_dim = out_dim
        self.n_conv_layers = n_conv_layers
        self.n_tasks = n_tasks
        self.target_hidden_dim = target_hidden_dim

    def forward(self, x, weights=None):
        x = F.conv2d(
            x,
            weight=weights["conv0.weights"].reshape(
                self.n_kernels, 1, self.kernel_size[0], self.kernel_size[0]
            ),
            bias=weights["conv0.bias"],
            stride=1,
        )
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        for i in range(1, self.n_conv_layers):
            x = F.conv2d(
                x,
                weight=weights[f"conv{i}.weights"].reshape(
                    int(2 ** i * self.n_kernels),
                    int(2 ** (i - 1) * self.n_kernels),
                    self.kernel_size[i],
                    self.kernel_size[i],
                ),
                bias=weights[f"conv{i}.bias"],
                stride=1,
            )
            x = F.relu(x)
            x = F.max_pool2d(x, 2)

        x = torch.flatten(x, 1)

        x = F.linear(
            x,
            weight=weights["hidden0.weights"].reshape(
                self.target_hidden_dim, x.shape[-1]
            ),
            bias=weights["hidden0.bias"],
        )

        logits = []
        for j in range(self.n_tasks):
            logits.append(
                F.linear(
                    x,
                    weight=weights[f"task{j}.weights"].reshape(
                        self.out_dim, self.target_hidden_dim
                    ),
                    bias=weights[f"task{j}.bias"],
                )
            )
        return logits
    def compute_l1_loss(self, w):
        return torch.square(w).sum()

In [4]:
@torch.no_grad()
def evaluate_hv(hypernet, targetnet, loader, rays, device):
    hypernet.eval()
    loss1 = nn.CrossEntropyLoss()
    loss2 = nn.CrossEntropyLoss()
    # num_samples = 0
    results = defaultdict(list)
    losses_all = []
    
    for ray in rays:
        total = 0.0
        task1_correct, task2_correct = 0.0, 0.0
        l1, l2 = 0.0, 0.0
        ray = torch.from_numpy(ray.astype(np.float32)).to(device)
        ray /= ray.sum()
        num_samples = 0
        losses = np.zeros(2)
        for batch in loader:
            hypernet.zero_grad()

            batch = (t.to(device) for t in batch)
            img, ys = batch
            bs = len(ys)
            num_samples += bs
            weights = hypernet(ray)
            logit1, logit2 = targetnet(img, weights)

            # loss
            curr_l1 = loss1(logit1, ys[:, 0])
            curr_l2 = loss2(logit2, ys[:, 1])
            losses_batch = [curr_l1.detach().cpu().tolist(),curr_l2.detach().cpu().tolist()]
            losses += bs * np.array(losses_batch)
        losses /= num_samples
        losses_all.append(losses)


    return losses_all

def train(
    train_dt,
    val_dt,
    test_dt,
    dataset,
    path,
    solver_type: str,
    epochs: int,
    hidden_dim: int,
    model: str,
    lr: float,
    wd: float,
    bs: int,
    val_size: float,
    n_rays: int,
    alpha: float,
    no_val_eval: bool,
    out_dir: str,
    device: torch.device,
    eval_every: int,
    model_type: str,
    resume: bool,
) -> None:
    # ----
    # Nets
    # ----
    if model == "lenet":
        if model_type == 'mlp':
            hnet = LeNetHyper([9, 5], ray_hidden_dim=hidden_dim)
            if resume:
                ckpt = torch.load(os.path.join(out_dir,'best_model_cheby_multi_'+str(dataset)+"_"+str(model_type)+'.pth'),map_location=device)
                hnet.load_state_dict(ckpt['state_dicts'])
            net = LeNetTarget([9, 5])
        elif model_type == 'trans_relu':
            hnet = LeNetHyper_trans([9, 5], ray_hidden_dim=hidden_dim,act_type='relu')
            if resume:
                ckpt = torch.load(os.path.join(out_dir,'best_model_cheby_multi_'+str(dataset)+"_"+str(model_type)+'_relu.pth'),map_location=device)
                hnet.load_state_dict(ckpt['state_dicts'])
            net = LeNetTarget_trans([9, 5])
        else:
            hnet = LeNetHyper_trans([9, 5], ray_hidden_dim=hidden_dim,act_type='gelu')
            if resume:
                ckpt = torch.load(os.path.join(out_dir,'best_model_cheby_multi_'+str(dataset)+"_"+str(model_type)+'_gelu.pth'),map_location=device)
                hnet.load_state_dict(ckpt['state_dicts'])
            net = LeNetTarget_trans([9, 5])

    logging.info(f"HN size: {count_parameters(hnet)}")

    hnet = hnet.to(device)
    net = net.to(device)

    # ---------
    # Task loss
    # ---------
    loss1 = nn.CrossEntropyLoss()
    loss2 = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(hnet.parameters(), lr=lr, weight_decay=wd)
    if solver_type == "epo":
        solver = EPOSolver(n_tasks=2, n_params=count_parameters(hnet))
    elif solver_type == "ls":
        # ls,cheby,utility
        solver = LinearScalarizationSolver()
    elif solver_type == "cheby":
        # ls,cheby,utility
        solver = ChebyshevBasedSolver(lower_bound = 0.1)
    elif solver_type == "utility":
        # ls,cheby,utility
        solver = UtilityBasedSolver(upper_bound = 200.0)

    # ----
    # data
    # ----
    #assert val_size > 0, "please use validation by providing val_size > 0"
#     train_loader, val_loader,test_loader = load_data(dataset,path)
    min_angle = 0.1
    max_angle = np.pi / 2 - 0.1
    test_rays = circle_points(n_rays, min_angle=min_angle, max_angle=max_angle)

    # ----------
    # Train loop
    # ----------
    epoch_iter = trange(epochs)
    best_hv_loss = -1 #-1
    start = time.time()

    for epoch in epoch_iter:
        losses_epoch = []
        loss_total = 0
        l1_total = 0
        l2_total = 0
        count = 0
        for i, batch in enumerate(train_loader):
            hnet.train()
            optimizer.zero_grad()
            img, ys = batch
            img = img.to(device)
            ys = ys.to(device)

            if alpha > 0:
                ray = torch.from_numpy(
                    np.random.dirichlet((alpha, alpha), 1).astype(np.float32).flatten()
                ).to(device)
            else:
                alpha = torch.empty(
                    1,
                ).uniform_(0.0, 1.0)
                ray = torch.tensor([alpha.item(), 1 - alpha.item()]).to(device)

            weights = hnet(ray)
            logit1, logit2 = net(img, weights)

            l1 = loss1(logit1, ys[:, 0])
            l2 = loss2(logit2, ys[:, 1])
            #print(l1.shape)
            losses = torch.stack((l1, l2))

            ray = ray.squeeze(0)
            loss = solver(losses, ray, list(hnet.parameters()))
            loss.backward()
            loss_total += loss.item()
            l1_total += l1.item()
            l2_total += l2.item()
            count += 1
            optimizer.step()
#         loss_hv = evaluate_hv(
#                     hypernet=hnet,
#                     targetnet=net,
#                     loader=val_loader,
#                     rays=test_rays,
#                     device=device,
#                 )
#         hv_loss = hypervolumn(np.array(loss_hv), type='loss', ref=np.ones(2) * 2)
#         if hv_loss>best_hv_loss:
#             best_hv_loss = hv_loss
#             epoch_iter.set_description(
#                 f"Update best model, Epoch: {epoch:.0f}, Hv_eval: {hv_loss:.5f}, total mean loss: {loss_total/count:.5f}, mean l1: {l1/count:.5f}, mean l2: {l2/count:.5f}"
#             )
#             # torch.save(hnet,os.path.join(out_dir,"best_model_"+str(solver_type)+"_multi_"+str(dataset)+".pt"))
#             save_dict = {'state_dicts': hnet.state_dict()}
#             if model_type == 'mlp':
#                 torch.save(save_dict,"best_model_"+str(solver_type)+"_multi_"+str(dataset)+"_"+str(model_type)+".pth")
#             elif model_type == 'trans_relu':
#                 torch.save(save_dict,"best_model_"+str(solver_type)+"_multi_"+str(dataset)+"_"+str(model_type)+".pth")
#             else:
#                 torch.save(save_dict,"best_model_"+str(solver_type)+"_multi_"+str(dataset)+"_"+str(model_type)+".pth")
#     ckpt = torch.load("/kaggle/working/best_model_"+str(solver_type)+"_multi_"+str(dataset)+"_"+str(model_type)+".pth")
#     hnet.load_state_dict(ckpt['state_dicts'])
    loss_hv_test = evaluate_hv(
                    hypernet=hnet,
                    targetnet=net,
                    loader=test_loader,
                    rays=test_rays,
                    device=device,
                )
    hv_loss_test = hypervolumn(np.array(loss_hv_test), type='loss', ref=np.ones(2) * 2)
    save_dict = {'state_dicts': hnet.state_dict()}
    torch.save(save_dict,"last_model_multi_"+str(dataset)+"_"+str(model_type)+"_"+str(hv_loss_test)+".pth")
    print("HV_test: ",hv_loss_test)
def load_data(dataset,data_path):
    # LOAD DATASET
    # ------------
    # MultiMNIST: multi_mnist.pickle
    if dataset == 'mnist':
        path = os.path.join(data_path,'multi_mnist.pickle')

    # MultiFashionMNIST: multi_fashion.pickle
    if dataset == 'fashion':
        path = os.path.join(data_path,'multi_fashion.pickle')

    # Multi-(Fashion+MNIST): multi_fashion_and_mnist.pickle
    if dataset == 'fashion_mnist':
        path = os.path.join(data_path,'multi_fashion_and_mnist.pickle')

    data = Dataset(path, val_size=0.1)
    train_set, val_set, test_set = data.get_datasets()
    batch_size = 256
    train_loader = torch.utils.data.DataLoader(
        dataset=train_set,
        batch_size=batch_size,num_workers=2,
        shuffle=True)
    val_loader = torch.utils.data.DataLoader(
        dataset=val_set,
        batch_size=batch_size,num_workers=2,
        shuffle=False)
    test_loader = torch.utils.data.DataLoader(
        dataset=test_set,
        batch_size=batch_size,num_workers=2,
        shuffle=False)
    return train_loader,val_loader, test_loader
def HPN_train(device,data_path,out_results,solver,batch_size,model_type,resume,train_loader, val_loader,test_loader):
    set_seed(42)
    set_logger()
#     datasets = ['mnist','fashion','fashion_mnist']
    if model_type == 'mlp':
        hidden_dim = 256
    else:
        hidden_dim = 256
#     for dataset in datasets:
    print("Dataset: ",dataset)
    print("Solver: ",solver)
    start = time.time()
    train(
        train_dt = train_loader, 
        val_dt = val_loader,
        test_dt = test_loader,
        dataset = dataset,
        path=data_path,
        solver_type=solver,
        epochs=150,
        hidden_dim=hidden_dim,
        model='lenet',
        lr=1e-4,
        wd=0.0,
        bs=batch_size,
        device=device,
        eval_every=10,
        no_val_eval=False,
        val_size=0.1,
        n_rays=25,
        alpha=0.2,
        out_dir=out_results,
        model_type = model_type,
        resume = resume,
    )
    end = time.time()
    print("Runtime training: ",end-start)

In [5]:
model_types = ["mlp", "trans_relu", "trans_gelu"]
datasets = ['mnist','fashion','fashion_mnist']
resume = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_path = "/kaggle/input/multi-mnist-fashion-fashmnist"
out_results = "/kaggle/input/hyper-trans/Hyper_trans/MTL/experiments/Multi_task/MultiMNIST/outputs"
hpn_solver = "epo"
batch_size = 256
hpn_solvers = ['cheby']

for dataset in datasets:
    train_loader, val_loader,test_loader = load_data(dataset,data_path)
    for hpn_solver in hpn_solvers:
        print("Solver: ",hpn_solver)
        for model_type in model_types:
            print("Model type: ",model_type)
            HPN_train(device,data_path,out_results,hpn_solver,batch_size,model_type,resume,train_loader, val_loader,test_loader)



Solver:  cheby
Model type:  mlp
Dataset:  mnist
Solver:  cheby


100%|██████████| 150/150 [09:51<00:00,  3.94s/it]


HV_test:  2.8076614946943352
Runtime training:  601.917332649231
Model type:  trans_relu
Dataset:  mnist
Solver:  cheby


100%|██████████| 150/150 [11:06<00:00,  4.44s/it]


HV_test:  2.839519136259348
Runtime training:  677.5950446128845
Model type:  trans_gelu
Dataset:  mnist
Solver:  cheby


...

HV_test:  2.8226038029601273
Runtime training:  609.4356753826141
Model type:  trans_relu
Dataset:  fashion_mnist
Solver:  cheby


100%|██████████| 150/150 [11:01<00:00,  4.41s/it]


HV_test:  2.830886944432782
Runtime training:  673.0524175167084
Model type:  trans_gelu
Dataset:  fashion_mnist
Solver:  cheby


100%|██████████| 150/150 [11:06<00:00,  4.44s/it]


HV_test:  2.755336216153385
Runtime training:  677.2304589748383


In [6]:
# from matplotlib import pyplot as plt
# def evaluate_hv(hypernet, targetnet, loader, rays, device):
#     hypernet.eval()
#     loss1 = nn.CrossEntropyLoss()
#     loss2 = nn.CrossEntropyLoss()
#     # num_samples = 0
#     losses_all = []
#     acc1, acc2 = [], []
#     for ray in rays:
#         total = 0.0
#         task1_correct, task2_correct = 0.0, 0.0
#         l1, l2 = 0.0, 0.0
#         ray = torch.from_numpy(ray.astype(np.float32)).to(device)
#         ray /= ray.sum()
#         num_samples = 0
#         losses = np.zeros(2)
#         for batch in loader:
#             hypernet.zero_grad()

#             batch = (t.to(device) for t in batch)
#             img, ys = batch
#             bs = len(ys)
#             num_samples += bs
#             weights = hypernet(ray)
#             logit1, logit2 = targetnet(img, weights)
#             curr_l1 = loss1(logit1, ys[:, 0])
#             curr_l2 = loss2(logit2, ys[:, 1])
#             losses_batch = [curr_l1.detach().cpu().tolist(),curr_l2.detach().cpu().tolist()]
#             losses += bs * np.array(losses_batch)
#             # acc
#             pred1 = logit1.data.max(1)[1]  # first column has actual prob.
#             pred2 = logit2.data.max(1)[1]  # first column has actual prob.
#             task1_correct += pred1.eq(ys[:, 0]).sum()
#             task2_correct += pred2.eq(ys[:, 1]).sum()
#         losses /= num_samples
#         acc_1 = task1_correct/num_samples
#         acc_2 = task2_correct/num_samples
#         acc1.append(acc_1.tolist())
#         acc2.append(acc_2.tolist())
#         losses_all.append(losses.tolist())

#     return losses_all, np.array(acc1), np.array(acc2)
# def circle_points(K, min_angle=None, max_angle=None):
#     # generate evenly distributed preference vector
#     ang0 = 1e-6 if min_angle is None else min_angle
#     ang1 = np.pi / 2 - ang0 if max_angle is None else max_angle
#     angles = np.linspace(ang0, ang1, K, endpoint=True)
#     x = np.cos(angles)
#     y = np.sin(angles)
#     return np.c_[x, y]
# def phn_test(dataset,path_data,save_weights,device,mode):
#     hnet = LeNetHyper([9, 5], ray_hidden_dim=265)
#     hnet1 = LeNetHyper_trans([9, 5], ray_hidden_dim=256,act_type = 'relu')
#     hnet2 = LeNetHyper_trans([9, 5], ray_hidden_dim=256,act_type = 'gelu')
#     # hnet3 = LeNetHyper([9, 5], ray_hidden_dim=hidden_dim)
#     # net = LeNetTarget([9, 5])

#     # Load dataset
#     # MultiMNIST: multi_mnist.pickle
#     if dataset == 'mnist':
#         path = os.path.join(path_data,'multi_mnist.pickle')

#     # MultiFashionMNIST: multi_fashion.pickle
#     if dataset == 'fashion':
#         path = os.path.join(path_data,'multi_fashion.pickle')

#     # Multi-(Fashion+MNIST): multi_fashion_and_mnist.pickle
#     if dataset == 'fashion_mnist':
#         path = os.path.join(path_data,'multi_fashion_and_mnist.pickle')

#     # Load checkpoints
#     ckpt = torch.load('/kaggle/working/best_model_ls_multi_'+str(dataset)+'_mlp.pth',map_location=device)
#     hnet.load_state_dict(ckpt['state_dicts'])
#     ckpt1 = torch.load('/kaggle/working/best_model_ls_multi_'+str(dataset)+'_trans_relu.pth',map_location=device)
#     hnet1.load_state_dict(ckpt1['state_dicts'])
#     ckpt2 = torch.load('/kaggle/working/best_model_ls_multi_'+str(dataset)+'_trans_gelu.pth',map_location=device)
#     hnet2.load_state_dict(ckpt2['state_dicts'])
#     # print(hnet2)
#     hnet = hnet.to(device)
#     hnet1 = hnet1.to(device)
#     hnet2 = hnet2.to(device)
#     fig = plt.figure()
#     min_angle = 0.01
#     max_angle = np.pi / 2 - min_angle
#     num_rays = 25
#     test_rays = circle_points(num_rays, min_angle=min_angle, max_angle=max_angle)
#     bs = 256
#     val_size = 0.1
#     data = Dataset(path, val_size=0.1)
#     train_set, val_set, test_set = data.get_datasets()
#     batch_size = 256
#     train_loader = torch.utils.data.DataLoader(
#         dataset=train_set,
#         batch_size=batch_size,num_workers=1,
#         shuffle=True)
#     val_loader = torch.utils.data.DataLoader(
#         dataset=val_set,
#         batch_size=batch_size,num_workers=1,
#         shuffle=False)
#     test_loader = torch.utils.data.DataLoader(
#         dataset=test_set,
#         batch_size=batch_size,num_workers=1,
#         shuffle=False)
#     loss_hv, acc_task_1_0, acc_task_2_0 = evaluate_hv(
#                         hypernet=hnet,
#                         targetnet=LeNetTarget([9, 5]),
#                         loader=test_loader,
#                         rays=test_rays,
#                         device=device,
#                     )
#     loss_hv1, acc_task_1_1, acc_task_2_1 = evaluate_hv(
#                         hypernet=hnet1,
#                         targetnet=LeNetTarget_trans([9, 5]),
#                         loader=test_loader,
#                         rays=test_rays,
#                         device=device,
#                     )
#     loss_hv2, acc_task_1_2, acc_task_2_2 = evaluate_hv(
#                         hypernet=hnet2,
#                         targetnet=LeNetTarget_trans([9, 5]),
#                         loader=test_loader,
#                         rays=test_rays,
#                         device=device,
#                     )
#     # loss_hv3, acc_task_1_3, acc_task_2_3 = evaluate_hv(
#     #                     hypernet=hnet3,
#     #                     targetnet=net,
#     #                     loader=test_loader,
#     #                     rays=test_rays,
#     #                     device=device,
#     #                 )

#     '''
#         Multi Mnist
#         x = [0.2,0.3,0.4,0.5]
#         y = [0.2,0.4,0.6,0.8]
#         plt.plot([0.2631959812361983,0.2631959812361983],[0.2,0.8],ls='-.',c='black',label='Single task')
#         plt.plot([0.2,0.5],[0.33708666510219815,0.33708666510219815],ls='-.',c='black')
#     '''
#     '''
#         Multi Fashion
#         x = [0.4,0.6,0.8,1.0]
#         y = [0.4,0.6,0.8,1.0]
#         plt.plot([0.4857283249686036,0.4857283249686036],[0.4,1],ls='-.',c='black',label='Single task')
#         plt.plot([0.4,1],[0.5331778043433081,0.5331778043433081],ls='-.',c='black')
#     '''
#     '''
#         Multi Fashion Mnist
#         x = [0.1,0.4,0.7,1.0]
#         y = [0.4,0.6,0.8,1.0]
#         plt.plot([0.16867540993645222,0.16867540993645222],[0.4,1],ls='-.',c='black',label = 'Single-task')
#         plt.plot([0.1,1],[0.44227917699874203,0.44227917699874203],ls='-.',c='black')
#     '''
#     '''
#         # Accuracy MNIST
#         x = [0.82,0.84,0.88,0.92]
#         y = [0.78,0.82,0.86,0.9]
#         plt.plot([0.91,0.91],[0.78,0.9],ls='-.',label = 'Single-task',c='black')
#         plt.plot([0.82,0.92],[0.885,0.885],ls='-.',c='black')   
#     '''
#     '''
#         # Accuracy Fashion
#         x = [0.68,0.73,0.78,0.83]
#         y = [0.6,0.68,0.76,0.84]
#         plt.plot([0.82,0.82],[0.6,0.84],ls='-.',label = 'Single-task',c='black')
#         plt.plot([0.68,0.83],[0.80,0.80],ls='-.',c='black')
#     '''
#     '''
#         # Accuracy Fashion + MNIST
#         x = [0.6,0.72,0.84,0.96]
#         y = [0.59,0.68,0.77,0.86]
#         plt.plot([0.94,0.94],[0.59,0.86],ls='-.',label = 'Single-task',c='black')
#         plt.plot([0.6,0.96],[0.84,0.84],ls='-.',c='black')
#     '''
#     if mode == "loss":
#         if dataset == 'mnist':
#             loss_hv = np.array(loss_hv)
#             loss_hv1 = np.array(loss_hv1)
#             loss_hv2 = np.array(loss_hv2)
#             # loss_hv3 = np.array(loss_hv3)
#             x_ = [loss_hv[0, 0],loss_hv[4, 0],loss_hv[9, 0],loss_hv[14, 0],loss_hv[19, 0],loss_hv[24, 0]]
#             y_ = [loss_hv[0, 1],loss_hv[4, 1],loss_hv[9, 1],loss_hv[14, 1],loss_hv[19, 1],loss_hv[24, 1]]
#             #plt.plot(loss_hv[:, 0], loss_hv[:, 1],label = 'HPN-LS',marker='*',linestyle = '-')
#             plt.plot(loss_hv[:, 0], loss_hv[:, 1],linestyle = '-',label = 'MLP')
#             plt.scatter(x_, y_,marker='*')
#             x1_ = [loss_hv1[0, 0],loss_hv1[4, 0],loss_hv1[9, 0],loss_hv1[14, 0],loss_hv1[19, 0],loss_hv1[24, 0]]
#             y1_ = [loss_hv1[0, 1],loss_hv1[4, 1],loss_hv1[9, 1],loss_hv1[14, 1],loss_hv1[19, 1],loss_hv1[24, 1]]
#             plt.plot(loss_hv1[:, 0], loss_hv1[:, 1],linestyle = '-.',label = 'Trans_ReLU')
#             plt.scatter(x1_, y1_,marker='o')
#             x2_ = [loss_hv2[0, 0],loss_hv2[4, 0],loss_hv2[9, 0],loss_hv2[14, 0],loss_hv2[19, 0],loss_hv2[24, 0]]
#             y2_ = [loss_hv2[0, 1],loss_hv2[4, 1],loss_hv2[9, 1],loss_hv2[14, 1],loss_hv2[19, 1],loss_hv2[24, 1]]
#             plt.plot(loss_hv2[:, 0], loss_hv2[:, 1],linestyle = ':',label = 'Trans_GeLU')
#             plt.scatter(x2_, y2_,marker='x')
#             # x3_ = [loss_hv3[0, 0],loss_hv3[4, 0],loss_hv3[9, 0],loss_hv3[14, 0],loss_hv3[19, 0],loss_hv3[24, 0]]
#             # y3_ = [loss_hv3[0, 1],loss_hv3[4, 1],loss_hv3[9, 1],loss_hv3[14, 1],loss_hv3[19, 1],loss_hv3[24, 1]]
#             # plt.plot(loss_hv3[:, 0], loss_hv3[:, 1],linestyle = '--')
#             # plt.scatter(x3_, y3_,marker='v',label = 'PHN-EPO')
#             x = [0.2,0.3,0.4,0.5]
#             y = [0.2,0.35,0.5,0.65]
#             plt.plot([0.2631959812361983,0.2631959812361983],[0.2,0.65],ls='-.',c='black',label='Single task')
#             plt.plot([0.2,0.5],[0.33708666510219815,0.33708666510219815],ls='-.',c='black')
#             plt.xlabel("Loss CE task left",fontsize=18)
#             plt.ylabel("Loss CE task right",fontsize=18)
#             plt.xticks(x)
#             plt.yticks(y)
#             plt.legend(fontsize=18)
#             plt.tight_layout()
#             plt.savefig('test_multi_'+str(dataset)+'.jpg')
#             #plt.savefig('test_multi_'+str(dataset)+'.pdf')
#         elif dataset == 'fashion':
#             loss_hv = np.array(loss_hv)
#             loss_hv1 = np.array(loss_hv1)
#             loss_hv2 = np.array(loss_hv2)
# #             loss_hv3 = np.array(loss_hv3)
#             x_ = [loss_hv[0, 0],loss_hv[4, 0],loss_hv[9, 0],loss_hv[14, 0],loss_hv[19, 0],loss_hv[24, 0]]
#             y_ = [loss_hv[0, 1],loss_hv[4, 1],loss_hv[9, 1],loss_hv[14, 1],loss_hv[19, 1],loss_hv[24, 1]]
#             #plt.plot(loss_hv[:, 0], loss_hv[:, 1],label = 'HPN-LS',marker='*',linestyle = '-')
#             plt.plot(loss_hv[:, 0], loss_hv[:, 1],linestyle = '-',label = 'MLP')
#             plt.scatter(x_, y_,marker='*')
#             x1_ = [loss_hv1[0, 0],loss_hv1[4, 0],loss_hv1[9, 0],loss_hv1[14, 0],loss_hv1[19, 0],loss_hv1[24, 0]]
#             y1_ = [loss_hv1[0, 1],loss_hv1[4, 1],loss_hv1[9, 1],loss_hv1[14, 1],loss_hv1[19, 1],loss_hv1[24, 1]]
#             plt.plot(loss_hv1[:, 0], loss_hv1[:, 1],linestyle = '-.',label = 'Transfomer')
#             plt.scatter(x1_, y1_,marker='o')
#             x2_ = [loss_hv2[0, 0],loss_hv2[4, 0],loss_hv2[9, 0],loss_hv2[14, 0],loss_hv2[19, 0],loss_hv2[24, 0]]
#             y2_ = [loss_hv2[0, 1],loss_hv2[4, 1],loss_hv2[9, 1],loss_hv2[14, 1],loss_hv2[19, 1],loss_hv2[24, 1]]
#             plt.plot(loss_hv2[:, 0], loss_hv2[:, 1],linestyle = ':',label = 'Trans_GeLU')
#             plt.scatter(x2_, y2_,marker='x')
# #             x3_ = [loss_hv3[0, 0],loss_hv3[4, 0],loss_hv3[9, 0],loss_hv3[14, 0],loss_hv3[19, 0],loss_hv3[24, 0]]
# #             y3_ = [loss_hv3[0, 1],loss_hv3[4, 1],loss_hv3[9, 1],loss_hv3[14, 1],loss_hv3[19, 1],loss_hv3[24, 1]]
# #             plt.plot(loss_hv3[:, 0], loss_hv3[:, 1],linestyle = '--')
# #             plt.scatter(x3_, y3_,marker='v',label = 'PHN-EPO')
#             x = [0.4,0.6,0.8,1.0]
#             y = [0.4,0.6,0.8,1.0]
#             plt.plot([0.4857283249686036,0.4857283249686036],[0.4,1],ls='-.',c='black',label='Single task')
#             plt.plot([0.4,1],[0.5331778043433081,0.5331778043433081],ls='-.',c='black')
#             plt.xlabel("Loss CE task left",fontsize=18)
#             plt.ylabel("Loss CE task right",fontsize=18)
#             plt.xticks(x)
#             plt.yticks(y)
#             plt.legend(fontsize=18)
#             plt.tight_layout()
#             plt.savefig('test_multi_'+str(dataset)+'.jpg')
#         elif dataset == 'fashion_mnist':
#             loss_hv = np.array(loss_hv)
#             loss_hv1 = np.array(loss_hv1)
#             loss_hv2 = np.array(loss_hv2)
# #             loss_hv3 = np.array(loss_hv3)
#             x_ = [loss_hv[0, 0],loss_hv[4, 0],loss_hv[9, 0],loss_hv[14, 0],loss_hv[19, 0],loss_hv[24, 0]]
#             y_ = [loss_hv[0, 1],loss_hv[4, 1],loss_hv[9, 1],loss_hv[14, 1],loss_hv[19, 1],loss_hv[24, 1]]
#             #plt.plot(loss_hv[:, 0], loss_hv[:, 1],label = 'HPN-LS',marker='*',linestyle = '-')
#             plt.plot(loss_hv[:, 0], loss_hv[:, 1],linestyle = '-',label = 'MLP')
#             plt.scatter(x_, y_,marker='*')
#             x1_ = [loss_hv1[0, 0],loss_hv1[4, 0],loss_hv1[9, 0],loss_hv1[14, 0],loss_hv1[19, 0],loss_hv1[24, 0]]
#             y1_ = [loss_hv1[0, 1],loss_hv1[4, 1],loss_hv1[9, 1],loss_hv1[14, 1],loss_hv1[19, 1],loss_hv1[24, 1]]
#             plt.plot(loss_hv1[:, 0], loss_hv1[:, 1],linestyle = '-.')
#             plt.scatter(x1_, y1_,marker='o',linestyle = '-.',label = 'Trans_ReLU')
#             x2_ = [loss_hv2[0, 0],loss_hv2[4, 0],loss_hv2[9, 0],loss_hv2[14, 0],loss_hv2[19, 0],loss_hv2[24, 0]]
#             y2_ = [loss_hv2[0, 1],loss_hv2[4, 1],loss_hv2[9, 1],loss_hv2[14, 1],loss_hv2[19, 1],loss_hv2[24, 1]]
#             plt.plot(loss_hv2[:, 0], loss_hv2[:, 1],linestyle = ':',label = 'Trans_GeLU')
#             plt.scatter(x2_, y2_,marker='x')
# #             x3_ = [loss_hv3[0, 0],loss_hv3[4, 0],loss_hv3[9, 0],loss_hv3[14, 0],loss_hv3[19, 0],loss_hv3[24, 0]]
# #             y3_ = [loss_hv3[0, 1],loss_hv3[4, 1],loss_hv3[9, 1],loss_hv3[14, 1],loss_hv3[19, 1],loss_hv3[24, 1]]
# #             plt.plot(loss_hv3[:, 0], loss_hv3[:, 1],linestyle = '--')
# #             plt.scatter(x3_, y3_,marker='v',label = 'PHN-EPO')
#             x = [0.1,0.4,0.7,1.0]
#             y = [0.4,0.6,0.8,1.0]
#             plt.plot([0.16867540993645222,0.16867540993645222],[0.4,1],ls='-.',c='black',label = 'Single-task')
#             plt.plot([0.1,1],[0.44227917699874203,0.44227917699874203],ls='-.',c='black')
#             plt.xlabel("Loss CE task left",fontsize=18)
#             plt.ylabel("Loss CE task right",fontsize=18)
#             plt.xticks(x)
#             plt.yticks(y)
#             plt.legend(fontsize=18)
#             plt.tight_layout()
#             plt.savefig('test_multi_'+str(dataset)+'.jpg')
#         print("HV MLP: ",hypervolumn(np.array(loss_hv), type='loss', ref=np.ones(2) * 2))
#         print("HV Trans_ReLU: ",hypervolumn(np.array(loss_hv1), type='loss', ref=np.ones(2) * 2))
#         print("HV Trans_GeLU: ",hypervolumn(np.array(loss_hv2), type='loss', ref=np.ones(2) * 2))
#         # print("HV PHN-EPO: ",hypervolumn(np.array(loss_hv3), type='loss', ref=np.ones(2) * 2))
# datasets = ['mnist','fashion','fashion_mnist']
# path_data = '/kaggle/input/multi-mnist-fashion-fashmnist'
# save_weights = './save_weights'
# device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# mode = 'loss'
# for dataset in datasets:
#     phn_test(dataset,path_data,save_weights,device,mode)
